In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
import os
import glob

import cv2
import numpy as np
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage.io import imread, imshow
from skimage.color import rgb2gray
from skimage.morphology import (erosion, dilation, closing, opening,
                                area_closing, area_opening)
from skimage.measure import label, regionprops, regionprops_table


In [2]:
print(os.path.dirname(os.path.abspath('Evtek_RC.ipynb')))

C:\Users\ralfy\Box\RC\Yoni\ralfy_assignment


In [3]:
# p = os.path.dirname(os.path.abspath('Evtek_RC.ipynb'))
# image_list = []
# for filename in glob.glob(p + '\CameraTop\*.png'):
    
#     im = cv2.imread(filename)
#     image_list.append(im)

In [4]:
p = os.path.dirname(os.path.abspath('Evtek_RC.ipynb'))

In [5]:
image_list = []
for filename in glob.glob(p + '\CameraTop\*.png'):
    
    im = cv2.imread(filename)
    image_list.append(im)

    
images = image_list.copy()

In [6]:
    
exr = []
for filename in glob.glob(p + '\CameraTop\**.exr'):
    
    exr_im = cv2.imread(filename,0)
    exr.append(exr_im)

In [35]:
path_list = [p + r"\CameraTop\main\cola_b",p + r"\CameraTop\main\fanta_b",p + r"\CameraTop\main\cherry_c",p + r"\CameraTop\main\coke_zero",
             p + r"\CameraTop\main\mthn_dew",p + r"\CameraTop\main\fanta_can",p + r"\CameraTop\main\cola_can"]
path = p + r"\CameraTop\separated"

if os.path.isdir(p + "\CameraTop\main") == False:
    os.mkdir(p + "\CameraTop\main")
    

if os.path.isdir(path) == False:
    os.mkdir(path)

for i,val in enumerate(path_list):
    
    if os.path.isdir(val): 
        continue
    else:
        os.mkdir(val)

In [45]:
def popimg(img):
    cv2.imshow("",img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    


 # exr iamges


exr_png = []
for id in range(len(exr)):
    im_exr = exr[id]
    im_exr=im_exr*65535
    im_exr[im_exr>65535]=65535
    im_exr=np.uint16(im_exr)
    exr_png.append(im_exr)
    # plt.imshow(im_exr)

class Hsv():
    def orange(self,img):
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        mask = cv2.inRange(hsv,(2, 50, 20), (25, 255, 255) )
        area = np.sum(mask == 255)
        return area
    def light_red(self,img):
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        mask = cv2.inRange(hsv,(0, 50, 20), (3, 255, 255) )
        area = np.sum(mask == 255)
        return area
    def dark_red(self,img):
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        mask = cv2.inRange(hsv,(170, 40, 20), (200, 255, 255) )
        area = np.sum(mask == 255)
        return area
    def cherry_red(self,img):
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        mask = cv2.inRange(hsv,(140, 20, 20), (180, 255, 255) )
        area = np.sum(mask == 255)
        return area
    def green(self,img):
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        mask = cv2.inRange(hsv,(40, 50, 20), (80, 255, 255) )
        area = np.sum(mask == 255)
        return area
    def diet(self,img):
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        mask = cv2.inRange(hsv,(2, 50, 20), (25, 255, 255) )
        area = np.sum(mask == 255)
        return area  
    
def separate(images,expng,path,it,path_list):

    painting = images

    gray_painting = cv2.cvtColor(images, cv2.COLOR_BGR2GRAY)
    # gray_painting = gray_painting[80:990,:]
    # imshow(painting)
    label_im = label(expng)
    regions = regionprops(label_im)
    # imshow(label_im)
    
    properties = ['area','convex_area','bbox_area', 'extent',  
              'mean_intensity', 'solidity', 'eccentricity', 
              'orientation']
    pd.DataFrame(regionprops_table(label_im, gray_painting, 
                 properties=properties))
    
    masks = []
    bbox = []
    list_of_index = []
    for num, x in enumerate(regions):
        area = x.area
        convex_area = x.convex_area
        if (num>=0 and (area>100) and (convex_area/area <5.55) #1.05
        and (convex_area/area >0.95)):
            masks.append(regions[num].convex_image)
            bbox.append(regions[num].bbox)   
            list_of_index.append(num)
    count = len(masks)
    
    # each1 = []
    fig, ax = plt.subplots(2, int(count/2))#, figsize=(15,8))
    plt.close()
    it = it
    for axis, box, mask in zip(ax.flatten(), bbox, masks):
        red  =  painting[:,:,0][box[0]:box[2], box[1]:box[3]] * mask
        green = painting[:,:,1][box[0]:box[2], box[1]:box[3]] * mask
        blue  = painting[:,:,2][box[0]:box[2], box[1]:box[3]] * mask
        image = np.dstack([red,green,blue])
        # each1.append(image)
        
        os.chdir(path)
        name = it
        filename = "%s.png" % name
        cv2.imwrite(filename, image)
        it += 1 
        # imshow(image)
        # axis.imshow(image)
        
        
        #HSV
        hsv_range = Hsv()
        # color1 = [hsv_range.orange(image),hsv_range.light_red(image),hsv_range.dark_red(image),hsv_range.cherry_red(image),hsv_range.green(image),hsv_range.diet(image)]
        color1 = [hsv_range.orange(image),hsv_range.light_red(image),hsv_range.dark_red(image),hsv_range.cherry_red(image),hsv_range.green(image)]
        
        # print(color1)
        max_idx = np.argmax(color1)
        # print(max_idx)
        
        if np.max(color1) >4200:
            #fanta bottle / can
            if max_idx==0:
                if 6000<np.max(color1) <10000:
                    cv2.imwrite(os.path.join(path_list[1] , filename), image)

                elif np.max(color1)>10500:
                    cv2.imwrite(os.path.join(path_list[5] , filename), image)

            #cola bottle
            elif max_idx == 1:
                cv2.imwrite(os.path.join(path_list[0], filename), image)

            # cola can
            elif max_idx==2:
                cv2.imwrite(os.path.join(path_list[6] , filename), image)

            # cola cherry bottle
            elif max_idx==3:
                # os.chdir(path_list[2])
                cv2.imwrite(os.path.join(path_list[2] ,filename), image)

            # mountain
            elif max_idx==4:
                # os.chdir(path_list[4])
                cv2.imwrite(os.path.join(path_list[4] , filename), image)

        # diet cola
        else:
            # os.chdir(path_list[3])
            cv2.imwrite(os.path.join(path_list[3] ,filename), image)
        
      
    return it


                           
it = 0

for idx, value in enumerate(images):
    it = separate(images[idx],exr_png[idx],path,it,path_list)

In [46]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense   
from sklearn.metrics import confusion_matrix
from keras import layers
import matplotlib.pyplot as plt

from keras.preprocessing.image import ImageDataGenerator

import numpy as np
import pandas as pd

In [50]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    p + r"\CameraTop\main",
    validation_split=0.2,
    subset="training",
    seed=1337,
    image_size=(28,28),
    batch_size=1
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    p + r"\CameraTop\main",
    validation_split=0.2,
    subset="validation",
    seed=1337,
    image_size=(28,28),
    batch_size=1
)

Found 1583 files belonging to 7 classes.
Using 1267 files for training.
Found 1583 files belonging to 7 classes.
Using 316 files for validation.


In [51]:
xtrain = []
ytrain = []

xtest = []
ytest = []

In [52]:
for images, labels in train_ds.take(len(train_ds)):
    for i in range(1):
        xtrain.append((images[i].numpy().astype("uint8")))
        ytrain.append(int(labels[i]))

In [53]:
for images, labels in val_ds.take(len(val_ds)):
    for i in range(1):
        xtest.append((images[i].numpy().astype("uint8")))
        ytest.append(int(labels[i]))

In [54]:

def frame(y):
    cherry_c = np.zeros(len(y))
    coke_zero =np.zeros(len(y))
    cola_b =  np.zeros(len(y))
    cola_can = np.zeros(len(y))
    fanta_b =  np.zeros(len(y))
    fanta_can =  np.zeros(len(y))
    mthn_dew =  np.zeros(len(y))

    for j,valj in enumerate(y):
        if valj == 0:
            cherry_c[j] = 1
        elif valj==1:
            coke_zero[j] = 2
        elif valj == 2:
            cola_b[j] = 3
        elif valj==4:
            cola_can[j] = 4
        elif valj==5:
            fanta_b[j] = 5
        elif valj==6:
            fanta_can[j] = 6
        elif valj==7:
            mthn_dew[j] =8

    data = {'cherry_c':cherry_c,
            'coke_zero':coke_zero,
            'cola_b':cola_b,
            'cola_can':cola_can,
            'fanta_b':fanta_b,
            'fanta_can':fanta_can,
            'mthn_dew':mthn_dew

           }
    return data
# Create DataFrame
df = pd.DataFrame(frame(ytrain))
df_test = pd.DataFrame(frame(ytest))

In [55]:
##Model structure
cnn3 = Sequential()
#layer 1
cnn3.add(keras.layers.Conv2D(32,kernel_size= (3, 3), activation='relu', kernel_initializer='he_normal', input_shape=(28, 28, 3)))
cnn3.add(keras.layers.BatchNormalization())

#Layer 2
cnn3.add(keras.layers.Conv2D(32,kernel_size= (3, 3), activation='relu'))
cnn3.add(keras.layers.BatchNormalization())
cnn3.add(keras.layers.MaxPooling2D((2, 2)))
cnn3.add(keras.layers.Dropout(0.6))

#Layer 4
cnn3.add(keras.layers.Conv2D(32,kernel_size= (3, 3), activation='relu'))
cnn3.add(keras.layers.BatchNormalization())


#Layer 3
cnn3.add(keras.layers.Conv2D(32,kernel_size= (3, 3), activation='relu'))
cnn3.add(keras.layers.MaxPooling2D((2, 2)))
cnn3.add(keras.layers.BatchNormalization())

#FC1
cnn3.add(keras.layers.Flatten())
cnn3.add(Dense(128, activation='relu'))
cnn3.add(keras.layers.BatchNormalization())
cnn3.add(Dense(64, activation='relu'))
cnn3.add(keras.layers.BatchNormalization())
cnn3.add(Dense(7, activation='softmax'))        

cnn3.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        896       
                                                                 
 batch_normalization (BatchN  (None, 26, 26, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 batch_normalization_1 (Batc  (None, 24, 24, 32)       128       
 hNormalization)                                                 
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 32)       0         
 )                                                               
                                                        

In [56]:
#compiling
optim = tf.keras.optimizers.Adam(learning_rate=0.001,name="adam")
cnn3.compile(optimizer=optim, loss='categorical_crossentropy', metrics=['accuracy'])
fit4 =cnn3.fit(np.array(xtrain),df,batch_size = 4,epochs = 100,validation_data = (np.array(xtest),np.array(df_test)),verbose=1)

Epoch 1/100
317/317 [==============================] - 4s 9ms/step - loss: 6.8102 - accuracy: 0.3496 - val_loss: 9.0508 - val_accuracy: 0.4051
Epoch 2/100
317/317 [==============================] - 3s 9ms/step - loss: 9.3589 - accuracy: 0.4743 - val_loss: 7.9900 - val_accuracy: 0.6108
Epoch 3/100
317/317 [==============================] - 3s 8ms/step - loss: 17.9975 - accuracy: 0.5154 - val_loss: 26.8403 - val_accuracy: 0.5380
Epoch 4/100
317/317 [==============================] - 3s 9ms/step - loss: 21.5898 - accuracy: 0.5296 - val_loss: 12.0518 - val_accuracy: 0.5348
Epoch 5/100
317/317 [==============================] - 3s 8ms/step - loss: 36.5749 - accuracy: 0.5225 - val_loss: 10.3860 - val_accuracy: 0.6709
Epoch 6/100
317/317 [==============================] - 3s 9ms/step - loss: 51.0436 - accuracy: 0.5185 - val_loss: 7.2415 - val_accuracy: 0.6741
Epoch 7/100
317/317 [==============================] - 3s 8ms/step - loss: 58.1185 - accuracy: 0.5241 - val_loss: 15.1990 - val_accurac

In [ ]:
plt.figure(figsize=(10, 6))
plt.title('Accuracy graph')
plt.plot(fit4.history['accuracy'])
plt.plot(fit4.history['val_accuracy'])
plt.ylabel('Train/Test Accuracy')
plt.xlabel('No. of Iterations')
plt.legend(['train', 'test'], loc='lower right')
# plt.savefig("Accuracy graph base.jpg")
plt.show()